In [1]:
%cd ../..

/home/kirill/PycharmProjects/pythonProject/RecoServiceTemplate


In [2]:
import pandas as pd
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender
from rectools.dataset import Dataset
from rectools import Columns
from rectools.models.popular import PopularModel
from models.userknn.userknn import UserKnn
from rectools.model_selection import TimeRangeSplit
from models.config import UserKnn_model

In [3]:
# Train model flag
RETRAIN = True

# Get KION dataset

In [4]:
interactions = pd.read_csv('{0}/interactions.csv'.format(UserKnn_model.dataset_path))
users = pd.read_csv('{0}/users.csv'.format(UserKnn_model.dataset_path))
items = pd.read_csv('{0}/items.csv'.format(UserKnn_model.dataset_path))

In [5]:
# rename columns, convert timestamp
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight},
                    inplace=True)

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

In [6]:
max_date = interactions['datetime'].max()
min_date = interactions['datetime'].min()

# From `itemkNN` to `userkNN `

In [7]:
n_folds = 3
unit = "D"
n_units = 4
periods = 2
freq = f"{n_units}{unit}"
last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)

date_range = pd.date_range(start=start_date, periods=periods, freq=freq,
                           tz=last_date.tz)
# generator of folds
cv = TimeRangeSplit(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [8]:
# we have just 1 test fold - no need to iterate over fold
(train_ids, test_ids, fold_info) = cv.split(interactions, collect_fold_stats=True).__next__()

# Fit userKNN model

In [9]:
train = interactions.loc[train_ids]
test = interactions.loc[test_ids]

In [10]:
model = TFIDFRecommender()

# Train model flag
if RETRAIN:
    userknn_model = UserKnn(model=model, N_users=50)
    userknn_model.fit(train)
else:
    userknn_model = UserKnn(model=model, N_users=50)
    userknn_model.load_weight(train, UserKnn_model.weight_path)

  0%|          | 0/850489 [00:00<?, ?it/s]

# Get recomendations

In [11]:
userknn_reco_df = userknn_model.predict(test, UserKnn_model.N_recs)

# Save recomendations df

In [12]:
userknn_reco_df.to_csv(UserKnn_model.save_reco_df_path, encoding='utf-8', index=False)

# df = pd.read_csv('userknn_reco_df', encoding='utf-8')


In [13]:
# # save model
import dill

with open(UserKnn_model.weight_path, 'wb') as f:
    dill.dump(userknn_model, f)